In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Activation, Dropout, Flatten, Dense
import  tensorflow.keras as K
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from imgaug import augmenters as iaa
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)

train_data_dir = "data/train"
validation_data_dir = "data/validation"
nb_train_samples = 387
nb_validation_samples = 30
batch_size = 16
epochs = 30

input = Input(shape=input_shape)

block1 = BatchNormalization(name='norm_0')(input)

# Block 1
block1 = Conv2D(8, (3,3), name='conv_11', activation='relu')(block1)
block1 = Conv2D(16, (3,3), name='conv_12', activation='relu')(block1)
block1 = Conv2D(32, (3,3), name='conv_13', activation='relu')(block1)
block1 = Conv2D(64, (3,3), name='conv_14', activation='relu')(block1)
block1 = MaxPooling2D(pool_size=(2, 2))(block1)
block1 = BatchNormalization(name='norm_1')(block1)

block1 = Conv2D(16, 1)(block1)

# Block 2
block2 = Conv2D(32, (3,3), name='conv_21', activation='relu')(block1)
block2 = Conv2D(64, (3,3), name='conv_22', activation='relu')(block2)
block2 = Conv2D(64, (3,3), name='conv_23', activation='relu')(block2)
block2 = Conv2D(128, (3,3), name='conv_24', activation='relu')(block2)
block2 = MaxPooling2D(pool_size=(2, 2))(block2)
block2 = BatchNormalization(name='norm_2')(block2)

block2 = Conv2D(64, 1)(block2)

# Block 3
block3 = Conv2D(64, (3,3), name='conv_31', activation='relu')(block2)
block3 = Conv2D(128, (3,3), name='conv_32', activation='relu')(block3)
block3 = Conv2D(128, (3,3), name='conv_33', activation='relu')(block3)
block3 = Conv2D(64, (3,3), name='conv_34', activation='relu')(block3)
block3 = MaxPooling2D(pool_size=(2, 2))(block3)
block3 = BatchNormalization(name='norm_3')(block3)

# Block 4
block4 = Conv2D(64, (3,3), name='conv_41', activation='relu')(block3)
block4 = Conv2D(32, (3,3), name='conv_42', activation='relu')(block4)
block4 = Conv2D(16, (3,3), name='conv_43', activation='relu')(block4)
block4 = Conv2D(8, (2,2), name='conv_44', activation='relu')(block4)
block4 = MaxPooling2D(pool_size=(2, 2))(block4)
block4 = BatchNormalization(name='norm_4')(block4)

block4 = Conv2D(2, 1)(block4)

block5 = GlobalAveragePooling2D()(block4)
output = Activation('softmax')(block5)

model = Model(inputs=[input], outputs=[output])
model.summary()
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation
sometimes = lambda aug: iaa.Sometimes(0.6, aug)
seq = iaa.Sequential([
                      iaa.GaussianBlur(sigma=(0 , 1.0)),
                      iaa.Sharpen(alpha=1, lightness=0),
                      iaa.CoarseDropout(p=0.1, size_percent=0.15),
                              sometimes(iaa.Affine(
                                                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                                                    translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                                    rotate=(-30, 30),
                                                    shear=(-16, 16)))
                    ])


train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=seq.augment_image,
    horizontal_flip=True,
    vertical_flip=True)



test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

checkpoint = ModelCheckpoint("f2.h5", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=0, mode='auto', cooldown=0, min_lr=0)

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, reduce_lr]
)

Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['glaucoma', 'not-glaucoma']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
norm_0 (BatchNormalization)  (None, 256, 256, 3)       12        
_________________________________________________________________
conv_11 (Conv2D)             (None, 254, 254, 8)       224       
_________________________________________________________________
conv_12 (Conv2D)             (None, 252, 252, 16)      1168      
_________________________________________________________________
conv_13 (Conv2D)             (None, 250, 250, 32)      4640      
_________________________________________________________________
conv_14 (Conv2D)             (None, 248, 248, 64)      18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 124, 124, 64)      0         
__________

24/24 [==============================] - 390s 16s/step - loss: 0.3303 - acc: 0.8646 - val_loss: 1.2399 - val_acc: 0.9231

Epoch 00020: acc did not improve from 0.89390
Epoch 21/30
24/24 [==============================] - 371s 15s/step - loss: 0.3257 - acc: 0.8756 - val_loss: 5.0369 - val_acc: 0.6875

Epoch 00021: acc did not improve from 0.89390
Epoch 22/30
24/24 [==============================] - 363s 15s/step - loss: 0.3228 - acc: 0.8808 - val_loss: 6.1993 - val_acc: 0.6154

Epoch 00022: acc did not improve from 0.89390
Epoch 23/30
24/24 [==============================] - 420s 17s/step - loss: 0.3013 - acc: 0.8880 - val_loss: 5.0369 - val_acc: 0.6875

Epoch 00023: acc did not improve from 0.89390
Epoch 24/30
24/24 [==============================] - 421s 18s/step - loss: 0.3295 - acc: 0.8697 - val_loss: 6.1993 - val_acc: 0.6154

Epoch 00024: acc did not improve from 0.89390
Epoch 25/30
24/24 [==============================] - 388s 16s/step - loss: 0.3296 - acc: 0.8704 - val_loss: 6.04